In [2]:
import pandas as pd
from IPython.display import clear_output
import pandas as pd

In [3]:
import time
import random

In [4]:
name_info = pd.read_csv('corrected_HS_Times.csv')
# name_info = pd.concat([df1,df2])
name_info.head()

,Unnamed: 0,50 Y Free Time,50 S Free Time,50 L Free Time,100 Y Free Time,100 S Free Time,100 L Free Time,200 Y Free Time,200 S Free Time,200 L Free Time,...,200 S Fly Points,200 L Fly Points,100 Y IM Points,100 S IM Points,200 Y IM Points,200 S IM Points,200 L IM Points,400 Y IM Points,400 S IM Points,400 L IM Points
0,880302,25.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,47.61,NaN,NaN,NaN,NaN,NaN
1,1653662,NaN,NaN,26.50,NaN,NaN,56.48,1:48.93,NaN,NaN,...,NaN,NaN,100.00,NaN,NaN,NaN,47.32,55.96,NaN,NaN
2,1653662,NaN,NaN,26.50,NaN,NaN,56.48,1:48.93,NaN,NaN,...,NaN,NaN,100.00,NaN,NaN,NaN,47.32,55.96,NaN,NaN
3,1131128,22.77,NaN,NaN,49.20,NaN,NaN,1:48.10,NaN,NaN,...,NaN,NaN,100.00,NaN,38.46,NaN,NaN,83.96,NaN,NaN
4,483789,22.71,NaN,NaN,51.62,NaN,NaN,NaN,NaN,2:45.24,...,NaN,NaN,94.93,NaN,100.00,NaN,NaN,NaN,NaN,NaN


In [7]:
# name_info = pd.read_parquet('./4more_name_info_dtypes.parquet')
name_info = name_info.rename(columns={'Unnamed: 0': 'ID'})
name_info.head()

,50 Y Free Time,50 S Free Time,50 L Free Time,100 Y Free Time,100 S Free Time,100 L Free Time,200 Y Free Time,200 S Free Time,200 L Free Time,500 Y Free Time,...,200 S Fly Points,200 L Fly Points,100 Y IM Points,100 S IM Points,200 Y IM Points,200 S IM Points,200 L IM Points,400 Y IM Points,400 S IM Points,400 L IM Points
ID,,,,,,,,,,,,,,,,,,,,,
880302,25.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,47.61,NaN,NaN,NaN,NaN,NaN
1653662,NaN,NaN,26.50,NaN,NaN,56.48,1:48.93,NaN,NaN,4:51.89,...,NaN,NaN,100.00,NaN,NaN,NaN,47.32,55.96,NaN,NaN
1653662,NaN,NaN,26.50,NaN,NaN,56.48,1:48.93,NaN,NaN,4:51.89,...,NaN,NaN,100.00,NaN,NaN,NaN,47.32,55.96,NaN,NaN
1131128,22.77,NaN,NaN,49.20,NaN,NaN,1:48.10,NaN,NaN,4:52.60,...,NaN,NaN,100.00,NaN,38.46,NaN,NaN,83.96,NaN,NaN
483789,22.71,NaN,NaN,51.62,NaN,NaN,NaN,NaN,2:45.24,NaN,...,NaN,NaN,94.93,NaN,100.00,NaN,NaN,NaN,NaN,NaN


In [8]:
# name_info = name_info.rename(index={'#': 368904})
# name_info.head(2)

In [9]:
# name_info.loc[368904,:]

In [10]:
name_info.isna().all(axis=1).sum()

0

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.display import clear_output

def get_swimmer_events(swimmer_id):
    url = f"https://www.swimcloud.com/swimmer/{swimmer_id}/powerindex/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
        'Referer': 'https://google.com/'
    }
    
    while True:
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"Failed to retrieve data for swimmer_id {swimmer_id}")
            print(swimmer_id)
            time.sleep(60)
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', class_='c-table-clean')
        
        if table is None:
            print("Table not found, skipping swimmer...")
            # time.sleep(7)
            return None

        rows = table.find('tbody').find_all('tr') if table.find('tbody') else None
        if rows is None:
            print("Rows not found, skipping swimmer...")
            # time.sleep(7)
            return None
        
        break

    data = {}
    for row in rows:
        cols = row.find_all('td')
        event = cols[0].text.strip()
        time_HS = cols[1].text.strip()
        points = cols[3].text.strip()
        data[event] = (time_HS, points)
    
    return data

def create_swimmer_dataframe(swimmer_ids):
    events = [
        "50 Y Free", "50 S Free", "50 L Free", "100 Y Free", "100 S Free", "100 L Free", "200 Y Free", "200 S Free", 
        "200 L Free", "500 Y Free", "400 S Free", "400 L Free", "1000 Y Free", "800 S Free", "800 L Free", "1650 Y Free", 
        "1500 S Free", "1500 L Free", "100 Y Back", "100 S Back", "100 L Back", "200 Y Back", "200 S Back", "200 L Back", 
        "100 Y Breast", "100 S Breast", "100 L Breast", "200 Y Breast", "200 S Breast", "200 L Breast", "100 Y Fly", 
        "100 S Fly", "100 L Fly", "200 Y Fly", "200 S Fly", "200 L Fly", "100 Y IM", "100 S IM", "200 Y IM", "200 S IM", 
        "200 L IM", "400 Y IM", "400 S IM", "400 L IM"
    ]
    
    columns = [f"{event} Time" for event in events] + [f"{event} Points" for event in events]
    result_df = pd.DataFrame(index=swimmer_ids, columns=columns)
    
    for i, swimmer_id in enumerate(swimmer_ids, 1):
        swimmer_data = get_swimmer_events(swimmer_id)
        if swimmer_data:
            for event in events:
                time_col = f"{event} Time"
                points_col = f"{event} Points"
                time_HS, points = swimmer_data.get(event, ('', ''))
                result_df.at[swimmer_id, time_col] = time_HS
                result_df.at[swimmer_id, points_col] = points
        
        if i % 10 == 0:
            print(f"Processed {i} swimmers...")

        # if i % 100 == 0:  # Save every 100 swimmers
            # result_df.to_parquet('progress_snapshot.parquet', engine='pyarrow')
        
        if i % 200 == 0:
            clear_output(wait=True)
            time.sleep(10)

    return result_df

# Load the existing DataFrame
# df = pd.read_parquet('./4more_name_info_dtypes.parquet', engine='pyarrow')  # Adjust the file path as needed
swimmer_ids = name_info.index[name_info.isna().all(axis=1)].tolist()

# Create the new DataFrame
new_df = create_swimmer_dataframe(swimmer_ids)
print("Done")

Done


In [13]:
# new_df.loc[~new_df.isna().all(axis=1)].iloc[0]

In [14]:
# new_df.loc[~new_df.isna().all(axis=1)].iloc[0]
# new_row = new_df.loc[~new_df.isna().all(axis=1)].iloc[0]

# # Get the index of the row to replace in name_info (assuming it's the same index as the new_row)
# index_to_replace = new_row.name
# print(index_to_replace)

# # Replace the row in name_info with the new_row
# name_info.loc[index_to_replace] = new_row

In [15]:
name_info.isna().all(axis=1).sum()

0

In [16]:
name_info.shape

(560, 88)

In [17]:
name_info = name_info.dropna(how='all')
name_info.shape

(560, 88)

In [18]:
name_info.to_csv('corrected_full_HS_times.csv',index=True)

In [19]:
name_info.head()

,50 Y Free Time,50 S Free Time,50 L Free Time,100 Y Free Time,100 S Free Time,100 L Free Time,200 Y Free Time,200 S Free Time,200 L Free Time,500 Y Free Time,...,200 S Fly Points,200 L Fly Points,100 Y IM Points,100 S IM Points,200 Y IM Points,200 S IM Points,200 L IM Points,400 Y IM Points,400 S IM Points,400 L IM Points
ID,,,,,,,,,,,,,,,,,,,,,
880302,25.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,47.61,NaN,NaN,NaN,NaN,NaN
1653662,NaN,NaN,26.50,NaN,NaN,56.48,1:48.93,NaN,NaN,4:51.89,...,NaN,NaN,100.00,NaN,NaN,NaN,47.32,55.96,NaN,NaN
1653662,NaN,NaN,26.50,NaN,NaN,56.48,1:48.93,NaN,NaN,4:51.89,...,NaN,NaN,100.00,NaN,NaN,NaN,47.32,55.96,NaN,NaN
1131128,22.77,NaN,NaN,49.20,NaN,NaN,1:48.10,NaN,NaN,4:52.60,...,NaN,NaN,100.00,NaN,38.46,NaN,NaN,83.96,NaN,NaN
483789,22.71,NaN,NaN,51.62,NaN,NaN,NaN,NaN,2:45.24,NaN,...,NaN,NaN,94.93,NaN,100.00,NaN,NaN,NaN,NaN,NaN
